In [ ]:
# visualization.py

from mesa.visualization.modules import NetworkModule, ChartModule
from mesa.visualization.ModularVisualization import ModularServer
from mesa.visualization.UserParam import Slider, Choice
from model import SocialModel, IndividualAgent, GenerativeAIAgent
import networkx as nx
import numpy as np
import nest_asyncio
nest_asyncio.apply()

# Define how agents are portrayed visually in the network
def agent_portrayal(agent):
    if isinstance(agent, IndividualAgent):
        # Represent the agent's knowledge by the size of the node
        size = 0.2 + 0.6 * (agent.knowledge / 100)  # Normalize knowledge for size (0 to 100)
        color = "green" if agent.knowledge > 50 else "blue"  # Agents with high knowledge are green
        
        return {
            "Shape": "circle",
            "r": size,  # Size based on knowledge
            "Color": color,
            "Layer": 1,
            "text": f"ID: {agent.unique_id}, K: {agent.knowledge:.1f}",
            "text_color": "black"
        }

# Define how the network nodes and edges are portrayed
def network_portrayal(G):
    portrayal = {
        "nodes": [],
        "edges": []
    }

    # Define the node attributes (agents)
    for node_id in G.nodes:
        agents = G.nodes[node_id].get("agent", [])
        for agent in agents:
            portrayal["nodes"].append({
                "id": node_id,
                "color": "blue" if agent.belief > 0 else "red",
                "size": 5,
                "tooltip": f"Agent ID: {agent.unique_id}\nBelief: {agent.belief:.2f}\nKnowledge: {agent.knowledge:.2f}"
            })

    # Define the edges
    for source, target in G.edges():
        portrayal["edges"].append({
            "source": source,
            "target": target,
            "color": 'black'
        })

    return portrayal
# Create the NetworkModule
network_module = NetworkModule(network_portrayal, 500, 500)

# Chart modules
chart_belief = ChartModule(
    [{"Label": "Average Belief", "Color": "Black"}],
    data_collector_name='datacollector'
)

chart_ai_bias = ChartModule(
    [{"Label": "AI Bias", "Color": "Red"}],
    data_collector_name='datacollector'
)

chart_knowledge = ChartModule(
    [{"Label": "Average Knowledge", "Color": "Blue"}],
    data_collector_name='datacollector'
)

chart_es = ChartModule(
    [{"Label": "Average Emotional State", "Color": "Green"}],
    data_collector_name='datacollector'
)

# Sliders and choice function for adjustable parameters
network_choice = Choice('Network Type', value='Small World',
    choices=['Random', 'Small World', 'Scale-Free']
)

num_agents_slider = Slider('Number of Agents', 100, 10, 200, 1
)

ai_accuracy_slider = Slider('AI Accuracy', 0.9, 0.5, 1.0, 0.01
)

ai_bias_slider = Slider('AI Bias', 0.1, 0, 1, 0.01
)

# Create the ModularServer to run the visualization
server = ModularServer(
    SocialModel,
    [network_module, chart_belief, chart_ai_bias, chart_knowledge, chart_es],
    "Social Model with Emotional States, Knowledge, and Beliefs",
    {
        "num_agents": num_agents_slider,
        "ai_accuracy": ai_accuracy_slider,
        "ai_bias": ai_bias_slider,
        "network_type": network_choice,
        "width": 10,
        "height": 10
    }
)

server.port = 8521  # Default Mesa port
server.launch()

